In [1]:
import pandas as pd
import io
import pymc3 as pm

In [2]:
with open('german.data', 'r') as f:
    data_str = f.read()


In [3]:
# reading file 
data = io.StringIO(data_str)
df = pd.read_csv(data,sep=' ', header=None)
df.columns = ['status', 'months', 'credit', 'purpose', 'amount', 'savings', 'employment', 'rate', 'sex', 'other', 'residence',
            'property', 'age', 'plan', 'housing', 'nb_credits', 'job', 'liability', 'phone', 'foreign', 'target']
df.loc[df['target']==1, 'target'] =0
df.loc[df['target']==2, 'target'] =1

In [1]:

lower=-10**8
higher=10**8
with pm.Model() as model:
    # define priors
    beta_0=pm.Uniform('beta_0', lower=lower, upper= higher)
    beta_age=pm.Uniform('beta_age', lower, higher)

    # define proba of belonging to class 1 
    p = pm.Deterministic('p', pm.math.sigmoid(beta_0+beta_age*df['age']))

with model:
    #fit the data 
    observed=pm.Bernoulli("target", p, observed=df['target'])
    start= pm.find_MAP()
    step= pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(2500, step=step, start=start)
    burned_trace=trace[15000:]
                      


NameError: name 'pm' is not defined